## Ejercicio Nro. 3 - Search engine con R

In [ ]:
# Bajar tweets con tema a elección
# Para ello deben crearse un usuario de twitter, y entrar en el modo desarrollador, para que les genere las claves para el token
# Agregar función de stemming para limpiar
# Correr función de búsqueda de cuatro palabras, tener en cuenta que esas cuatro palabras tienen que tener stemming tambien. 

## Librerías

In [2]:
# Instalación
packages <- c("httpuv", "rtweet", "ggplot2", "dplyr", "tidytext", "purrr", "stringr", "tm")
if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

# Cargo librerías
library("httpuv")
library("rtweet")
library("ggplot2")
library("dplyr")
library("tidytext")
library("purrr")
library("stringr")
library("tm")

## Conexión a Twitter

In [16]:
## Buscar sus claves y copiarlas
api_key <- "8F7a0IwVrOUXEoHAqDmd..."
api_secret_key <- "NpuRVqrFLSel9WF54xIKUI3qph6LACF8DKYt9N0PbdMLZl..."

## authenticate via web browser
token <- create_token(
  app = 'txmUTN',
  consumer_key = api_key,
  consumer_secret = api_secret_key)

## Scraping de tweets

In [4]:
rt <- search_tweets(q ="vicentin", n = 20000, include_rts = FALSE)
head(rt, 5)

"Rate limit exceeded - 88"Warning message:
"Rate limit exceeded"

user_id,status_id,created_at,screen_name,text,source,display_text_width,reply_to_status_id,reply_to_user_id,reply_to_screen_name,...,statuses_count,favourites_count,account_created_at,verified,profile_url,profile_expanded_url,account_lang,profile_banner_url,profile_background_url,profile_image_url
718070828563349505,1275161447941713926,2020-06-22 20:19:15,DellatorreRaul,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",Twitter for Android,150,NA,NA,NA,...,1738,1076,2016-04-07 13:40:01,FALSE,https://t.co/rUxTWPZjwd,http://motoreconomico.com.ar/,NA,https://pbs.twimg.com/profile_banners/718070828563349505/1583780814,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1109755274624200705/A9e9TXn8_normal.jpg
718070828563349505,1275036346260885509,2020-06-22 12:02:08,DellatorreRaul,No ES SOLO VICENTIN. El río Paraná esta privatizado. Es una caja negra. Por allí salen u$s 28.000 millones de exportaciones anuales en granos y oleaginosas al margen de la vista del Estado. Todos los años este complejo transnacionalizado le evade al fisco 1500 millones de uss.,Twitter for Android,279,NA,NA,NA,...,1738,1076,2016-04-07 13:40:01,FALSE,https://t.co/rUxTWPZjwd,http://motoreconomico.com.ar/,NA,https://pbs.twimg.com/profile_banners/718070828563349505/1583780814,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1109755274624200705/A9e9TXn8_normal.jpg
718070828563349505,1275037131463892992,2020-06-22 12:05:15,DellatorreRaul,NO SOLO ES VICENTIN. Todos los años este complejo transnacionalizado le evade al fisco u$s1500 millones. Los puertos son de ellos. Los elevadores de granos de ellos. La concesión de la hidrovía de ellos. No tenemos como controlarlos. Es eso lo que está en disputa. No Vicentìn,Twitter for Android,276,NA,NA,NA,...,1738,1076,2016-04-07 13:40:01,FALSE,https://t.co/rUxTWPZjwd,http://motoreconomico.com.ar/,NA,https://pbs.twimg.com/profile_banners/718070828563349505/1583780814,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1109755274624200705/A9e9TXn8_normal.jpg
166206368,1275161416190832641,2020-06-22 20:19:07,agustynn77,@g_alonsoshell @clarincom Cuando Vicentin robo al Estado?,Twitter for Android,31,1275149939954368513,1248868691225333760,g_alonsoshell,...,656,2999,2010-07-13 16:05:15,FALSE,NA,NA,NA,NA,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1256753874293194752/CXoiNif5_normal.jpg
767020227280379913,1275161394896408577,2020-06-22 20:19:02,leoaguirrec,"Si salen a la calle por la cuarentena y vicentin, imaginate cuando volvamos a fase 1...",Twitter for Android,87,NA,NA,NA,...,9940,1435,2016-08-20 15:27:27,FALSE,NA,NA,NA,https://pbs.twimg.com/profile_banners/767020227280379913/1471709877,NA,http://pbs.twimg.com/profile_images/1246909002480574465/0kh-6Zg-_normal.jpg


In [5]:
# Cantidad de tweets bajados
doc <- rt  %>% select(status_id,text)
dim(doc)

[1] 17980     2

## Limpieza de tweets

### Función para limpiar tweets

In [6]:
limpiar <- function(texto){
  # El orden de la limpieza no es arbitrario
    # Se convierte todo el texto a minúsculas
    nuevo_texto <- tolower(texto)
    # Eliminación de páginas web (palabras que empiezan por "http." seguidas 
    # de cualquier cosa que no sea un espacio)
    nuevo_texto <- str_replace_all(nuevo_texto,"http\\S*", "")
    # Eliminación de signos de puntuación
    nuevo_texto <- str_replace_all(nuevo_texto,"[[:punct:]]", " ")
    # Eliminación de números
    nuevo_texto <- str_replace_all(nuevo_texto,"[[:digit:]]", " ")
    # Eliminación de espacios en blanco múltiples
    nuevo_texto <- str_replace_all(nuevo_texto,"[\\s]+", " ")
    # Elimino palabras vacías
    nuevo_texto <- removeWords(nuevo_texto, stopwords('es'))
    
    return(nuevo_texto)
}

In [7]:
doc <- doc %>% mutate(texto_limpio = map(.x = text, .f = limpiar))
head(doc, 3)

status_id,text,texto_limpio
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate
1275036346260885509,No ES SOLO VICENTIN. El río Paraná esta privatizado. Es una caja negra. Por allí salen u$s 28.000 millones de exportaciones anuales en granos y oleaginosas al margen de la vista del Estado. Todos los años este complejo transnacionalizado le evade al fisco 1500 millones de uss.,solo vicentin río paraná privatizado caja negra allí salen u$s millones exportaciones anuales granos oleaginosas margen vista años complejo transnacionalizado evade fisco millones uss
1275037131463892992,NO SOLO ES VICENTIN. Todos los años este complejo transnacionalizado le evade al fisco u$s1500 millones. Los puertos son de ellos. Los elevadores de granos de ellos. La concesión de la hidrovía de ellos. No tenemos como controlarlos. Es eso lo que está en disputa. No Vicentìn,solo vicentin años complejo transnacionalizado evade fisco u$s millones puertos elevadores granos concesión hidrovía controlarlos disputa vicentìn


### Stemming

In [8]:
stemming <- function(texto){
    nuevo_texto <- stemDocument(texto, language = "spanish")
    return(nuevo_texto)
}

doc <- doc %>% mutate(texto_stemming = map(.x = texto_limpio, .f = stemming ))
head(doc[3:4])

texto_limpio,texto_stemming
vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,vicentin debat polit gobiern pedr peretti siempr argentin vicentin offshor debat
solo vicentin río paraná privatizado caja negra allí salen u$s millones exportaciones anuales granos oleaginosas margen vista años complejo transnacionalizado evade fisco millones uss,sol vicentin rio paran privatiz caj negr alli sal u$s millon export anual gran oleagin marg vist años complej transnacionaliz evad fisc millon uss
solo vicentin años complejo transnacionalizado evade fisco u$s millones puertos elevadores granos concesión hidrovía controlarlos disputa vicentìn,sol vicentin años complej transnacionaliz evad fisc u$s millon puert elev gran concesion hidrov control disput vicentìn
g alonsoshell clarincom vicentin robo,g alonsoshell clarincom vicentin rob
si salen calle cuarentena vicentin imaginate volvamos fase,si sal call cuarenten vicentin imaginat volv fas
caba zona influencia vuelve fase decir hacer videitos tik tok publicar recetas facebook cantar himno balcones cada aplaudir cacerolear depende sector pertenezcas gritar vicentin campo demás,cab zon influenci vuelv fas dec hac videit tik tok public recet facebook cant himn balcon cad aplaud cacerol depend sector pertenezc grit vicentin camp demas


## División de la columna de tweets en tokens 

In [9]:
doc_frame <- doc  %>% unnest_tokens(word, texto_stemming)
head(doc_frame)

status_id,text,texto_limpio,word
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,vicentin
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,debat
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,polit
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,gobiern
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,pedr
1275161447941713926,"VICENTIN: ""NO HAY UN DEBATE POLITICO EN EL GOBIERNO"". Pedro Peretti en Siempre es ... https://t.co/qQQ7wfZeyh #Argentina #Vicentin #offshorear #debate",vicentin debate politico gobierno pedro peretti siempre argentina vicentin offshorear debate,peretti


## Búsqueda de tweets relevantes

### Función de búsqueda de tweets (ranking de palabras)

In [10]:
wordsearch <- function(a=NULL, b=NULL, c=NULL, d=NULL) {
   
    # Count of each word and the name of the document. 
    docwords <- doc_frame %>%  count(status_id, word, sort = TRUE) %>% ungroup()
   
    # Calculate the tf-idf of each word, then order the result according to tf-idf.
    docrank <- data.frame(bind_tf_idf(docwords, word, status_id, n))
    docrank <- docrank[order(docrank$tf_idf, decreasing = TRUE), ]
    
    # Filtering the results according to the search queries.
    results <- docrank[docrank$word %in% c(a, b, c, d), ]
   
    # Here we rank the search results according to sum of the values of
    # the weighting of the words they contain that match the search queries.
    results <- aggregate(results$tf_idf, list(results$status_id), sum)
    results <- results[order(results$x, decreasing = TRUE), ]
    
    # Add the document titles to the search results and return the top 15
    # most relevant documents.
    Titles <- (list(results$Group.1[1:15]))
    Result_Rank <- c(1:15)
    search_results <- data.frame(Titles, Result_Rank)
    names(search_results) <- c('Document', 'Ranks')
    
    # Print the results
    print(search_results)
}

In [11]:
# Se eligen 4 palabras y se aplica stemming
palabras <- c("juez","expropiación","campo", "vicentin")

palabras <- stemDocument(palabras, language = "spanish")
palabras

[1] "juez"     "expropi"  "camp"     "vicentin"

In [12]:
# Se realiza una búsqueda de los tweets más relevantes
wordsearch(palabras)

              Document Ranks
1  1275010602499538944     1
2  1274824631816519681     2
3  1274712217708048385     3
4  1274721913739317249     4
5  1274593480552198145     5
6  1274673931367206913     6
7  1274584382796451841     7
8  1275119752554467331     8
9  1274708498643910657     9
10 1274729485510811648    10
11 1274826203854245889    11
12 1274883349522382855    12
13 1274890615097081862    13
14 1274728611040432128    14
15 1274744264526888962    15


In [13]:
# Tweet más relevante
resultado <- rt  %>% filter(status_id == '1275010602499538944')
resultado[1:5]

user_id,status_id,created_at,screen_name,text
176225971,1275010602499538944,2020-06-22 10:19:50,franciscomingor,@_LauraDiMarco La traición al campo. https://t.co/MWOW8H2QEc


In [14]:
# Segundo tweet más relevante
resultado <- rt  %>% filter(status_id == '1274824631816519681')
resultado[1:5]

user_id,status_id,created_at,screen_name,text
189414110,1274824631816519681,2020-06-21 22:00:51,PATBARI,Cuidemos TODOS al Juez de #Vicentin https://t.co/LEHpg1sTyR


In [15]:
# Tercer tweet más relevante
resultado <- rt  %>% filter(status_id == '1274712217708048385')
resultado[1:5]

user_id,status_id,created_at,screen_name,text
1978143390,1274712217708048385,2020-06-21 14:34:10,Alguienqueve,el JUEZ Fabian Lorenzini : https://t.co/gebrzXu9gh
